In [20]:
# https://account.sendinblue.com/advanced/api

from __future__ import print_function
import time
import requests
import sib_api_v3_sdk
from sib_api_v3_sdk.rest import ApiException
from pprint import pprint

# chargement de la clef API via dotenv

from dotenv import load_dotenv
load_dotenv()
SIB_API_KEY = os.environ.get('SIB_API_KEY')

In [21]:
# Get all the contacts : https://developers.sendinblue.com/reference/getcontacts-1

def get_all_contacts(apikey, limit = 50, offset = 0, modified_since = '2020-09-20T19:20:30+01:00', sort = 'desc'):
    """Récupère l'intégralité des contacts du compte

    Parameters
    ----------
    apikey : str
        API key
    limit : int64
        Number of documents per page
    offset : int64
        Index of the first document in the page
    modified_since : string (UTC date-time : 'YYYY-MM-DDTHH:mm:ss.SSSZ')
        Filter (urlencoded) the contacts modified after a given UTC date-time (YYYY-MM-DDTHH:mm:ss.SSSZ).
    sort : str ('asc' or 'desc')
        Sort the results in the ascending/descending order of record creation. Default order is descending if sort is not passed

    Returns
    -------
    contacts : Json
        Liste de tous les contacts correspondant à la requête

    """
    configuration = sib_api_v3_sdk.Configuration()
    configuration.api_key['api-key'] = apikey
    api_instance = sib_api_v3_sdk.ContactsApi(sib_api_v3_sdk.ApiClient(configuration))
    try:
        api_response = api_instance.get_contacts(limit=limit, offset=offset, modified_since=modified_since, sort = sort)
        pprint(api_response)
    except ApiException as e:
        print("Exception when calling ContactsApi->get_contacts: %s\n" % e)


get_all_contacts(SIB_API_KEY, modified_since = '2022-01-01T00:00:00+01:00')

{'contacts': [{'attributes': {'NOM': 'Turcato', 'PRENOM': 'Nicolas'},
               'createdAt': '2022-01-04T17:22:19.000+01:00',
               'email': 'nicolas.turcato@etudiant.univ-lr.fr',
               'emailBlacklisted': False,
               'id': 238,
               'listIds': [15],
               'modifiedAt': '2022-01-07T14:21:52.075+01:00',
               'smsBlacklisted': False},
              {'attributes': {'NOM': 'Bonneau', 'PRENOM': 'Yohann'},
               'createdAt': '2022-01-04T17:21:44.000+01:00',
               'email': 'bonneauyo@gmail.com',
               'emailBlacklisted': False,
               'id': 237,
               'listIds': [15],
               'modifiedAt': '2022-01-07T14:21:44.587+01:00',
               'smsBlacklisted': False},
              {'attributes': {'NOM': 'Bataille', 'PRENOM': 'Charlotte'},
               'createdAt': '2022-01-04T17:21:05.000+01:00',
               'email': 'charlotte.bataille@etudiant.univ-lr.fr',
               'emailBl

In [23]:
# Get all the lists : https://developers.sendinblue.com/reference/getlists-1

def get_all_lists(apikey, limit = 50, offset = 0):
    """Récupère l'intégralité des listes du compte

    Parameters
    ----------
    apikey : str
        API key
    limit : int64
        Number of documents per page
    offset : int64
        Index of the first document in the page
    sort : str ('asc' or 'desc')
        Sort the results in the ascending/descending order of record creation. Default order is descending if sort is not passed

    Returns
    -------
    count : int64
        Nombre de listes
    lists : Json
        Liste de toutes les listes suivant la requête

    """
    configuration = sib_api_v3_sdk.Configuration()
    configuration.api_key['api-key'] = apikey
    api_instance = sib_api_v3_sdk.ListsApi(sib_api_v3_sdk.ApiClient(configuration))

    try:
        api_response = api_instance.get_lists(limit=limit, offset=offset)
        pprint(api_response)
    except ApiException as e:
        print("Exception when calling ListsApi->get_lists: %s\n" % e)

get_all_lists(SIB_API_KEY)



{'count': 15,
 'lists': [{'folderId': 13,
            'id': 19,
            'name': 'manif_interet_univ',
            'totalBlacklisted': 0,
            'totalSubscribers': 0,
            'uniqueSubscribers': 10},
           {'folderId': 11,
            'id': 18,
            'name': 'testeurs_prospects',
            'totalBlacklisted': 0,
            'totalSubscribers': 0,
            'uniqueSubscribers': 7},
           {'folderId': 1,
            'id': 17,
            'name': 'testeurs_vip',
            'totalBlacklisted': 0,
            'totalSubscribers': 0,
            'uniqueSubscribers': 6},
           {'folderId': 1,
            'id': 16,
            'name': 'relance_post_onboarding',
            'totalBlacklisted': 0,
            'totalSubscribers': 0,
            'uniqueSubscribers': 31},
           {'folderId': 1,
            'id': 15,
            'name': 'nouveaux_arrivants',
            'totalBlacklisted': 0,
            'totalSubscribers': 0,
            'uniqueSubscribers